In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

 ** Device Configuration **

In [2]:
#CUDA Availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [3]:
# Data Collection and preprocessing
data = load_breast_cancer()
X = data.data
y = data.target


In [4]:
# TRAIN DATA
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [5]:
#Strandardize the Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# convert data to pytorch tensor
X_train = torch.tensor(X_train,dtype=torch.float32).to(device)
X_test = torch.tensor(X_test,dtype=torch.float32).to(device)
y_train = torch.tensor(y_train,dtype=torch.float32).to(device)
y_test = torch.tensor(y_test,dtype=torch.float32).to(device)

Neural Network Architecture

In [7]:
class BreastCancerClassifier(nn.Module):

    def __init__(self,input_size,hidden_size,output_size):
      super(BreastCancerClassifier,self).__init__()
      self.fc1 = nn.Linear(input_size,hidden_size)
      self.relu = nn.ReLU()
      self.fc2 = nn.Linear(hidden_size,output_size)
      self.sigmoid = nn.Sigmoid()
    def forward(self,x):
      out = self.fc1(x)
      out = self.relu(out)
      out = self.fc2(out)
      out = self.sigmoid(out)
      return out


In [8]:
#hyperparameter
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100


In [9]:
#initialize the NN
model = BreastCancerClassifier(input_size,hidden_size,output_size).to(device)

In [10]:
#loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

Training the Neural Network

In [12]:
for epoch in range(num_epochs):
    model.train()

    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #calculate accuracy
    with torch.no_grad():
        predicted = outputs.round()
        correct = (predicted == y_train.view(-1,1)).float().sum()
        accuracy = correct/y_train.size(0)
    if (epoch+1)%10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy.item()*100:.2f}%')

Epoch [10/100], Loss: 0.0965, Accuracy: 97.58%
Epoch [20/100], Loss: 0.0902, Accuracy: 98.02%
Epoch [30/100], Loss: 0.0848, Accuracy: 98.46%
Epoch [40/100], Loss: 0.0802, Accuracy: 98.46%
Epoch [50/100], Loss: 0.0761, Accuracy: 98.46%
Epoch [60/100], Loss: 0.0726, Accuracy: 98.46%
Epoch [70/100], Loss: 0.0694, Accuracy: 98.46%
Epoch [80/100], Loss: 0.0666, Accuracy: 98.68%
Epoch [90/100], Loss: 0.0640, Accuracy: 98.68%
Epoch [100/100], Loss: 0.0617, Accuracy: 98.68%


Model Evaluation

In [15]:
#eval on training set
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1,1)).float().sum()
    accuracy = correct/y_train.size(0)
    print(f'Training Accuracy: {accuracy.item()*100:.2f}%')

Training Accuracy: 98.68%


In [16]:
#test data
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    predicted = outputs.round()
    correct = (predicted == y_test.view(-1,1)).float().sum()
    accuracy = correct/y_test.size(0)
    print(f'Testing Accuracy: {accuracy.item()*100:.2f}%')

Testing Accuracy: 99.12%
